In [1]:
# debut du div, americain, avancer sur convergence

from pricing import BlackScholesPricer, TrinomialTree, Market, Option
from pricing import convergence

# market = Market(S0=100, r=0.05, sigma=0.2)
# option = Option(K=103, option_type="call", T=1.0, option_class="european")
# tree = TrinomialTree(market, option, N=2)

# price = tree.root.price_recursive(option)
# print("Prix de l’option:", price)

# bs_price = BlackScholesPricer().price(S=market.S0, K=option.K, T=option.T, r=market.r, q=0, sigma=market.sigma, option_type='call') # q=0 car pas de dividendes
# print("Black-Scholes Price: {:.4f}".format(bs_price))


# convergence.bs_convergence_by_strike(market, option, [80, 90, 100, 110, 120, 130, 140, 150, 160], N=100)

# convergence.bs_convergence_by_step(market, option, bs_price, max_n = 1000, step = 100)

ImportError: cannot import name 'Market' from partially initialized module 'pricing' (most likely due to a circular import) (/Users/yves-mariesaliou/Documents/Cours/M2 IEF 272/INFO QUANT/PROJET/PRICER V1/src/pricing/__init__.py)

In [3]:
tree.root.next_up.next_up.proba

np.float64(0.012210932698873014)